In [ ]:
import os
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

import sys
sys.path.append(os.getenv("PROJECT_PATH"))

from inference import get_model
import supervision as sv

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

BALL_DETECTION_MODEL_ID = 'football-players-detection-3zvbc/2'
BALL_DETECTION_MODEL = get_model(BALL_DETECTION_MODEL_ID, ROBOFLOW_API_KEY)

FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/14"
FIELD_DETECTION_MODEL = get_model(FIELD_DETECTION_MODEL_ID, ROBOFLOW_API_KEY)

SOURCE_VIDEO_PATH = "../../data/videos/3-fullvideo.mp4"
#SOURCE_VIDEO_PATH = "../../data/videos/testvid.mp4"

In [ ]:
#Frame collection for training
from tqdm import tqdm

frames = []

for frame_index, frame in enumerate(tqdm(sv.get_video_frames_generator(SOURCE_VIDEO_PATH), desc="Processing...")):
    if frame_index % 5 == 0: #default: 5
        frames.append(frame)

print(f"Total frames collected: {len(frames)}")

In [8]:
#Full tracking
import supervision as sv
import numpy as np
from utils.viewtransformer import ViewTransformer
from utils.pitchconfig import SoccerPitchConfiguration

CONFIG = SoccerPitchConfiguration()

BALL_ID = 0

ball_coords = []

# Tracker
tracker = sv.ByteTrack()

for frame_index, frame in enumerate(tqdm(frames, desc="Processing...")):
    tqdm.write(f"... frame {frame_index}: ")
    
    # ball and players detections
    result = BALL_DETECTION_MODEL.infer(frame, confidence=0.2)[0]
    detections = sv.Detections.from_inference(result)

    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)
    
    # Detect pitch key points
    result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(result)
    
    # Ensure key_points is not None and contains valid data
    if key_points is None or key_points.confidence is None:
        print(f"Skipping frame {frame_index} due to missing key points.")
        continue  # Skip this frame and move to the next one

    # project ball, players and gk on pitch
    filter = key_points.confidence[0] > 0.5
    frame_reference_points = key_points.xy[0][filter]
    pitch_reference_points = np.array(CONFIG.vertices)[filter]

    # Ensure there are at least 4 points for homography calculation
    if len(frame_reference_points) < 4 or len(pitch_reference_points) < 4:
        print(f"Skipping frame due to insufficient keypoints: {len(frame_reference_points)} found.")
        continue  # Skip this frame and move to the next one

    # Proceed with homography calculation
    transformer = ViewTransformer(
        source = frame_reference_points,
        target = pitch_reference_points
    )

    frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)

    # --- Save all ball coordinates in pitch space ---
    if len(pitch_ball_xy) > 0:
        coords = [(float(x), float(y)) for x, y in pitch_ball_xy]
        ball_coords.append((frame_index, coords))
    else:
        ball_coords.append((frame_index, []))

... frame 439: 


... frame 440: 


... frame 441: 


... frame 442: 


... frame 443: 


...

... frame 541: 


... frame 542: 


... frame 543: 


... frame 544: 


Processing...: 100%|██████████| 545/545 [24:08<00:00,  2.66s/it]


In [ ]:
#Save coordinates per frame to CSV
import csv
import numpy as np
import os

video_name = os.path.splitext(os.path.basename(SOURCE_VIDEO_PATH))[0]

output_csv_path = f"../../data/raw-coords/ball-{video_name}.csv" 

ball_lookup = {f: balls for f, balls in ball_coords}

header = (
    ["frame_index", "balls"]
)

all_frames = sorted(set(
    list(ball_lookup.keys())
))

with open(output_csv_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for frame_index in all_frames:
        row = [frame_index]
        # Balls
        balls = ball_lookup.get(frame_index, [])
        if balls:
            # Join all ball coordinates as "x1,y1;x2,y2;..."
            balls_str = ";".join([f"{x:.2f},{y:.2f}" for x, y in balls])
            row.append(balls_str)
        else:
            row.append("")
        writer.writerow(row)

print(f"Coordinates per frame saved to {output_csv_path}")

Coordinates per frame saved to ../../data/raw-coords/ball-3-fullvideo.csv


: 